In [ ]:
# On installe les bibliothèques nécessaires, notamment pour le TFT et l'agent RL
!python -m pip install --upgrade pip setuptools wheel
!pip install pandas numpy scikit-learn tensorflow gymnasium stable-baselines3 google-colab pytorch_lightning pytorch-forecasting pandas-ta pytorch-lightning -q


# ------------------------------------------------------------------------------
# SECTION 0 : INSTALLATION ET IMPORTATION DES BIBLIOTHÈQUES
# ------------------------------------------------------------------------------


In [ ]:
!pip install google-colab

In [ ]:
import pandas as pd
import numpy as np
import glob
from google.colab import drive

from sklearn.preprocessing import MinMaxScaler
import pandas_ta as ta

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss

import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import DQN


In [ ]:

# --- Monter Google Drive pour accéder aux données ---
drive.mount('/content/drive')

# ==============================================================================
# PARTIE 1 : CHARGEMENT ET INGÉNIERIE DES CARACTÉRISTIQUES (FEATURES)
# ==============================================================================
print("\n--- PARTIE 1 : Chargement et préparation des données ---")

# --- 1.1 Chargement et combinaison des fichiers CSV ---
folder_path = '/content/drive/MyDrive/Global Stock Market (2008-2023)/*.csv' # Adaptez le chemin si nécessaire
all_files = glob.glob(folder_path)
df_list = [pd.read_csv(filename) for filename in all_files]
df = pd.concat(df_list, ignore_index=True)

# --- 1.2 Filtrage et nettoyage initial ---
ticker_valide = '^GSPC'  # S&P 500
df_filtered = df[df['Ticker'] == ticker_valide].copy()
print(f"Nombre de lignes trouvées pour le ticker '{ticker_valide}': {len(df_filtered)}")

# Garder les colonnes essentielles et gérer les dates
df_processed = df_filtered[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
df_processed['Date'] = pd.to_datetime(df_processed['Date'])
df_processed.sort_values('Date', inplace=True)
df_processed.reset_index(drop=True, inplace=True)

# --- 1.3 Ingénierie des caractéristiques pour le TFT ---
print("Ajout des indicateurs techniques et caractéristiques temporelles...")
# Indicateurs techniques via pandas-ta
df_processed.ta.sma(length=20, append=True)  # Moyenne mobile simple
df_processed.ta.rsi(length=14, append=True)  # Relative Strength Index
df_processed.ta.bbands(length=20, append=True) # Bandes de Bollinger

# Caractéristiques basées sur la date
df_processed['day_of_week'] = df_processed['Date'].dt.dayofweek
df_processed['month'] = df_processed['Date'].dt.month
df_processed['year'] = df_processed['Date'].dt.year

# Gérer les valeurs NaN créées par les indicateurs (remplissage vers l'arrière)
df_processed.fillna(method='bfill', inplace=True)

# ==============================================================================
# PARTIE 2 : PRÉDICTION AVEC LE TEMPORAL FUSION TRANSFORMER (TFT)
# ==============================================================================
print("\n--- PARTIE 2 : Prédiction avec le Temporal Fusion Transformer (TFT) ---")

# --- 2.1 Formatage des données pour Pytorch Forecasting ---
print("Formatage des données pour le TFT...")
df_processed['time_idx'] = range(len(df_processed))
df_processed['group'] = ticker_valide # Identifiant de la série temporelle

# --- 2.2 Création du TimeSeriesDataSet ---
max_encoder_length = 60  # Utiliser les 60 derniers jours
max_prediction_length = 1 # Pour prédire le jour suivant

# Séparer les données pour l'entraînement et la validation du TFT
training_cutoff = df_processed['time_idx'].max() - (max_prediction_length * 365) # Garder 1 an pour validation

training_data = TimeSeriesDataSet(
    df_processed[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Close",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["group"],
    time_varying_known_reals=["time_idx", "day_of_week", "month", "year"],
    time_varying_unknown_reals=[
        'Open', 'High', 'Low', 'Volume', 'SMA_20', 'RSI_14',
        'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0'
    ],
    target_normalizer=GroupNormalizer(groups=["group"], transformation="softplus"),
)

validation_data = TimeSeriesDataSet.from_dataset(training_data, df_processed, predict=True, stop_randomization=True)

# Création des DataLoaders
batch_size = 128
train_dataloader = training_data.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
val_dataloader = validation_data.to_dataloader(train=False, batch_size=batch_size, num_workers=2)

# --- 2.3 Entraînement du modèle TFT ---
print("Entraînement du modèle TFT...")
pl.seed_everything(42)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
trainer = pl.Trainer(
    max_epochs=30,
    accelerator="gpu",
    gradient_clip_val=0.1,
    limit_train_batches=50,
    callbacks=[early_stop_callback],
)
tft = TemporalFusionTransformer.from_dataset(
    training_data,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=16,
    loss=QuantileLoss(),
    optimizer="Ranger"
)
trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# --- 2.4 Génération des prédictions et enrichissement du DataFrame ---
print("Génération des prédictions du TFT pour tout le dataset...")
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

full_dataloader = validation_data.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=2)
raw_predictions, x = best_tft.predict(full_dataloader, mode="raw", return_x=True)

# On extrait la prédiction médiane (quantile 0.5, qui est à l'indice 3)
predictions = raw_predictions["prediction"][:, :, 3].numpy().flatten()

# On crée un DataFrame avec les prédictions et on le fusionne
tft_predictions_df = pd.DataFrame({
    'tft_prediction': predictions
}, index=df_processed.index[df_processed['time_idx'].isin(x['decoder_time_idx'].flatten())])

df_final = df_processed.join(tft_predictions_df)
df_final.dropna(inplace=True) # Supprimer les premières lignes sans prédictions
df_final.set_index('Date', inplace=True)

print("Aperçu du DataFrame final enrichi :")
print(df_final.head())


# ==============================================================================
# PARTIE 3 : APPRENTISSAGE PAR RENFORCEMENT AVEC DQN
# ==============================================================================
print("\n--- PARTIE 3 : Apprentissage par Renforcement avec DQN ---")

# --- 3.1 Définition de l'environnement de Trading ---
class TradingEnv(gym.Env):
    def __init__(self, data: pd.DataFrame, initial_balance=10_000):
        super().__init__()
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.action_space = spaces.Discrete(3) # 0: Hold, 1: Buy, 2: Sell
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(len(self.data.columns) + 2,), dtype=np.float32
        )

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.balance = self.initial_balance
        self.shares_held = 0
        self.net_worth = self.initial_balance
        self.current_step = 0
        return self._get_observation(), {}

    def _get_observation(self):
        # On inclut les données du marché, la balance et les actions détenues
        obs = np.concatenate([
            self.data.iloc[self.current_step].values,
            [self.balance],
            [self.shares_held]
        ]).astype(np.float32)
        return obs

    def step(self, action):
        previous_net_worth = self.net_worth
        current_price = self.data['Close'].iloc[self.current_step]

        if action == 1 and self.balance > current_price: # Buy
            self.shares_held += 1
            self.balance -= current_price
        elif action == 2 and self.shares_held > 0: # Sell
            self.shares_held -= 1
            self.balance += current_price

        self.net_worth = self.balance + (self.shares_held * current_price)

        # RÉCOMPENSE AMÉLIORÉE : changement de la valeur nette depuis l'étape précédente
        reward = self.net_worth - previous_net_worth

        self.current_step += 1
        done = self.current_step >= len(self.data) - 1

        return self._get_observation(), reward, done, False, {}

    def render(self, mode='human'):
        print(f'Step: {self.current_step}, Net Worth: {self.net_worth:.2f}, Shares: {self.shares_held}, Balance: {self.balance:.2f}')

# --- 3.2 Entraînement de l'agent DQN ---
print("Entraînement de l'agent DQN...")
# Séparer les données pour l'entraînement et le test de l'agent RL
train_size_rl = int(len(df_final) * 0.8)
train_data_rl = df_final.iloc[:train_size_rl]
test_data_rl = df_final.iloc[train_size_rl:]

# Instancier l'environnement d'entraînement
env_train = TradingEnv(data=train_data_rl)

# Créer et entraîner le modèle DQN avec plus d'exploration
model_dqn = DQN(
    "MlpPolicy",
    env_train,
    verbose=1,
    tensorboard_log="./dqn_tft_tensorboard/",
    exploration_fraction=0.5,
    exploration_final_eps=0.05,
    learning_rate=0.0005,
    buffer_size=100_000
)
model_dqn.learn(total_timesteps=200_000, progress_bar=True)
model_dqn.save("dqn_tft_trader")

# ==============================================================================
# PARTIE 4 : ÉVALUATION ET BACKTESTING
# ==============================================================================
print("\n--- PARTIE 4 : Évaluation sur les données de test ---")

# --- 4.1 Lancer le backtesting ---
env_test = TradingEnv(data=test_data_rl)
obs, info = env_test.reset()
done = False

print("\n--- Début du Backtesting ---")
while not done:
    action, _states = model_dqn.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env_test.step(action)
    done = terminated or truncated
    env_test.render()

# --- 4.2 Calculer le ROI final ---
final_net_worth = env_test.net_worth
initial_balance = env_test.initial_balance
roi = ((final_net_worth - initial_balance) / initial_balance) * 100

print("\n--- Fin du Backtesting ---")
print(f"Bilan Initial : {initial_balance:.2f} $")
print(f"Bilan Final   : {final_net_worth:.2f} $")
print(f"Retour sur Investissement (ROI) final : {roi:.2f}%")